# Игра "5 Букв"
Тинькофф перезапустил игру "5 букв", в которой нужно отгадывать слова. Вам нужно набрать существительное из пяти букв, система покажет какие буквы этого слова есть в загаданном слове, на основе этой информации вы должны отгадать слово за несколько попыток.  
Для призов банк подготовил скидки, кэшбэки и др. Например, за 60 слов можно получить 50%-й кэшбэк в Ozon.
Играем в игру из приложения Тиньков банка "5 букв".  Правила игры похожи на игру Wordle.
<div>
<img src="game.jpeg" width="200"/>
</div>   
Попробуем придумать какую-нибудь стратегию для выигрыша.

## Чтение файла

Для нашей игры прочитаем список слов, которые:
* состоят из 5 букв
* не содержат символы не из алфавита (дефисы, английские буквы)
* не начинаются с заглавной буквы  

Исходный файл [nouns.csv](https://github.com/Badestrand/russian-dictionary/blob/master/nouns.csv) взят с гитхаба (https://github.com/Badestrand/russian-dictionary) (Creative Commons Attribution Share Alike 4.0).

In [41]:
words = []
with open('nouns.txt', encoding='utf-8') as file:
    words = [w.strip() for w in file.readlines()]

words = [w for w in words if not 'ё' in w] # отбрасываем слова с буквой "ё" 
words.sort()
(words[:10], len(words))

(['аарон',
  'абака',
  'аббат',
  'абвер',
  'абзац',
  'аборт',
  'абрек',
  'абрис',
  'абхаз',
  'абцуг'],
 3607)

## Класс WordPredictor

In [42]:
class WordPredictor:
    def __init__(self, words, takeplace=True):
        """
        words     : массив слов
        takeplace : Учитывать место буквы при подсчете рейтинга слова. 
                    Например встречаемость буквы й в зависимости от места: [0, 1, 38, 8, 50]
        """
        self.takeplace = takeplace
        self.words = words.copy() # TODO: проверка, что бы все слова были одинаковой длины
        self.alphabet = ''
        
        # для расчета рейтинга слова соберем статистику по буквам
        dict={}

        for w in self.words:
            for l in w:
                if not l in self.alphabet:
                    self.alphabet += l
                dict[l] = dict.get(l, 0) + 1
        
        sorted_tuples = sorted(dict.items(), key=lambda x: x[1],reverse=True)
        self.letter_rating = {k: v for k, v in sorted_tuples}
        mostusedletters = "".join(list(self.letter_rating.keys()))
        self.alphabet = ''.join(sorted(self.alphabet))
        print(f"Алфавит: {self.alphabet}")
        print(f"Буквы, отсортированные по степени встречаемости: {mostusedletters}")
        
        # а теперь посмотрим, на каких местах каждая буква встречается чаще
        self.letter_places={k: [0]*5 for k in self.alphabet}
        for w in self.words:
            for l,i in zip(w, range(5)):        
                self.letter_places[l][i] += 1
         
        self.reset()
        
                
    def _score(self, w, ignore=''):
        """
        Подсчет рейтинга слова
        ignore     : Строка с метками для, определяющая, учитывать букву при расчете рейтинга слова (символ "_") или нет.
                     Это нужно, что бы считать рейтинг слов, для которых уже угаданы позиции букв. 
                     В этом случае угаданные буквы игнорируются в рейтинге
        """
        if ignore == '': ignore = '_'*len(w) # если ignore не задана, то считаем рейтинг по всем буквам
        s=0
        for i in range(len(w)):
            if ignore[i] not in self.alphabet: # вместо буквы находится символ, значит буква учавствует в подсчете
                if not self.takeplace:
                    s += self.sorted_dict[w[i]]//w.count(w[i])
                else:
                    s += self.letter_places[w[i]][i]//w.count(w[i])
        return s
    
    def reset(self):
        """
        сброс попыток подбора слова
        """
        self.wrongletters=''               # список букв, которых вообще нет в угадываемом слове
        self.wrongplaces={}                # словарь букв, которые есть в слове, но не на своём месте
        self.doubleletters=''              # список повторяющихся в слове букв
        self.correctplaces='_'*len(words[0]) # список угаданных букв
        self.attempts = []                 # список попыток с результатом
        self.goodwords = words.copy()      # список слов, которые удовлетворяют условиям
        self.endgame = False               # флаг окончания поиска
        
    
    def _scan_results(self, testword, result):
        """
        Разбор результата проверки с записью в соответствующие переменные
        testword : слово, которое проверялось
        result   : результат проверки, "-" - буквы в слове нет, "*" - буква есть, но на другом месте
                   "+" - буква угадана
        """
        # TODO: проверка на равенство длин testword, result и words
        for i, (r, l) in enumerate(zip(result, testword)):
            if (r=='-'):
                if not l in self.correctplaces: # условие нужно, что бы не записать в wrongletters букву, которая уже угадана
                    self.wrongletters = ''.join(set(self.wrongletters) | set(l))
            elif (r=='*'):
                self.wrongplaces[l] = self.wrongplaces.get(l,[]) + [i]
                if l in self.correctplaces:  # если буква уже угадана, и она найдена снова
                    self.doubleletters += l
            else:
                t = list(self.correctplaces)
                t[i] = l
                self.correctplaces = ''.join(t)
                

    def _is_good_word(self, word):
        """
        проверка слова по условиям wrongletters, wrongplaces, correctplaces, doubleletters
        """
        
        # 1) в слове должны быть найденные буквы в нужных местах
        for testletter, goodletter in zip(word, self.correctplaces):
            if goodletter != '_': # если угаданная буква есть на этом месте
                if testletter != goodletter: # и она не равна проверяемой букве
                    return False # сразу нет

        # 2) в слове не должно быть отсутствующих букв
        for letter in word:
            if letter in self.wrongletters:
                return False
            
        # 3) если есть одинаковые буквы, то они толжны быть в слове
        for doubleletter in self.doubleletters:
            if word.count(doubleletter) < 2:
                return False

        # что будет, если в слове будут одинаковые буквы?
        # если проверяемое слово содержит одинаковые буквы, и места будут не угаданы (символ *), то оба неугаданных места будут сохранены в словаре wrongplaces
        # одна буква будет угадана и она будет в списке correctplaces
        # а возможное место второй буквы можно будет вычислить по словарю wrongplaces
        # проверки 4) и 5) можно оставить, так как они не противоречат наличию одинаковых букв в слове
            
        # 4) в слове должны быть неправильно расположенные буквы
        for letter in list(self.wrongplaces.keys()): # берем каждую неправильно расположенную букву
            if letter not in word: # неправильно расположенные буквы должны быть в слове (странно, да?)
                return False
            
        # 5) если буква есть в списке неправильно расположенных букв, то её место не должно быть в списке неправильных мест
        for i, testletter in enumerate(word):
            if i in self.wrongplaces.get(testletter, []): # буква есть в списке и её место тоже есть в списке
                return False
                
        return True # если слово прошло все проверки

    def _find_good_words(self):
        """
        Вывод списка слов, удовлетворяющих условиям в переменных wrongletters, wrongplaces, correctplaces
        """
        goodwordsnext = []

        for w in self.goodwords:
            if self._is_good_word(w):
                goodwordsnext += [w]
        return goodwordsnext
        
    def predict_next_words(self, testword, result):
        """
        главная функция предсказания следующих слов
        testword  :  слово, которое было указано в приложении
        result    :  результат предсказания этого слова
        """
        if self.endgame:
            print('Нужно сбросить поиск вызвав reset()')
            return None, None
        
        self.attempts.append([testword, result]) # добавляем попытку в массив попыток (зачем?)
        self._scan_results(testword, result)      # сканируем результат попытки
        nextwords = self._find_good_words()  # ищем слова, подходящие под условия
        
        if len(nextwords) == 0:
            print('Что-то пошло не так. Не могу найти слова, попадающие под условия')
            self.endgame = True
            return None, (None, None)
        elif len(nextwords) == 1:
            # print(f'Слово найдено: {nextwords[0]}.\nЧисло ходов {len(self.attempts)}.')
            self.endgame = True
            return "1 из 1:", [[nextwords[0], self._score(nextwords[0])]]
        else:
            self.goodwords = nextwords
            scorelist=[[w, self._score(w, self.correctplaces)] for w in self.goodwords] # считаем рейтинг слов...
            newwords=sorted(scorelist,key=lambda x: x[1],reverse=True) # и сортируем по убыванию
            return f"{min(10, len(newwords))} из {len(newwords)}:", newwords[:10]
    
    @staticmethod
    def check_word(correctword, testword):
        """
        проверка слова, как это присходит в игре
        """
        tresult=''
        correctletters = ''
        for rl, tl in zip(correctword, testword):
            if tl not in correctword:
                tresult += '-'
            else:
                if tl==rl:
                    tresult += '+'
                    correctletters += tl # собираем угаданыне буквы
                else: 
                    tresult += '*'
                    
        # теперь пройдемся еще раз по результатам сканирования и уберем в результатах звёздочки для тех букв, места которых точно определены (плюсики)
        # вместо check_word('чайка', 'кадка') = '*+-++' нужно получить '-+-++', потому что в слове "чайка" буква "к" одна
        result = ''
        for tl, r in zip(testword, tresult):
            replace = False
            if r == '*':                         # если буква есть в слове, но не на своем месте
                if correctword.count(tl) == 1:   # и в загаданном слове эта буква только одна
                    if tl in correctletters:     # и место этой буквы точно определено
                        replace = True
                                   
            result += '-' if replace else r # меняем '*' на '-'
        return result
    

    def find_word(self, secret, tryword='порка'):
        """
        поиск загаданного слова secret, используюя описанные в классе алгоритмы
        """
        self.reset()
    
        i = 0
        while i < 10:
            i += 1
            checkresult = WordPredictor.check_word(secret, tryword) # проверяем слово
            _, nextwords = self.predict_next_words(tryword, checkresult) # ищем новые слова
            print( (tryword, checkresult, self.correctplaces, len(nextwords)) )
            if(len(nextwords)==1):
                print(f'Конец игры. Загаданное слово: {nextwords[0][0]}')
                break
            else:
                tryword=nextwords[0][0]

## Проверка алгоритма поиска

In [43]:
wp = WordPredictor(words)

Алфавит: абвгдежзийклмнопрстуфхцчшщъыьэюя
Буквы, отсортированные по степени встречаемости: аокриетнлсупмбвдгзяшьыхчфжйцющэъ


In [44]:
wp.find_word('чайка')

('порка', '---++', '___ка', 10)
('силка', '---++', '___ка', 10)
('метка', '---++', '___ка', 10)
('банка', '-+-++', '_а_ка', 10)
('кадка', '-+-++', '_а_ка', 10)
('шавка', '-+-++', '_а_ка', 5)
('качка', '-+*++', '_а_ка', 1)
Конец игры. Загаданное слово: чайка


In [45]:
wp.wrongletters

'лнеспимдбвротш'

## Пример поиска слова "манеж"

<div>
<img src="example.png" width="200"/>  
</div>  
1) Начинаем со слова "порка":

In [46]:
wp.reset() # сбрасываем поиск
wp.predict_next_words('порка', '----*')

('10 из 290:',
 [['сатин', 1972],
  ['салун', 1847],
  ['балет', 1839],
  ['валет', 1822],
  ['ватин', 1812],
  ['талия', 1771],
  ['бадин', 1750],
  ['талес', 1746],
  ['баден', 1709],
  ['шатен', 1700]])

2\) Пусть следующее слово будет "сатин"

In [47]:
wp.predict_next_words('сатин', '-+--*')

('10 из 18:',
 [['манеж', 787],
  ['ганец', 773],
  ['шанец', 753],
  ['ханец', 714],
  ['надел', 705],
  ['манул', 687],
  ['мазня', 662],
  ['набег', 653],
  ['башня', 623],
  ['навал', 532]])

3\) "Манеж" наиболее вероятное слово и оно же загадано. Но что будет, если выбрать, например, "ганец"?

In [48]:
wp.predict_next_words('ганец', '-+++-') 

('1 из 1:', [['манеж', 1526]])

Остаётся только "манеж". Парам-парам-па, пиу!

### Отгадываем слово "гладь"

In [49]:
wp.reset() # сбрасываем поиск
wp.predict_next_words('порка', '----*')

('10 из 290:',
 [['сатин', 1972],
  ['салун', 1847],
  ['балет', 1839],
  ['валет', 1822],
  ['ватин', 1812],
  ['талия', 1771],
  ['бадин', 1750],
  ['талес', 1746],
  ['баден', 1709],
  ['шатен', 1700]])

In [50]:
wp.predict_next_words('сатин', '-*---')

('10 из 26:',
 [['шемая', 1182],
  ['бугай', 1127],
  ['дувал', 1109],
  ['вуаль', 1082],
  ['шугай', 1039],
  ['чувал', 1036],
  ['легаш', 1033],
  ['вылаз', 1029],
  ['чуваш', 970],
  ['гуашь', 906]])

In [51]:
wp.predict_next_words('бугай', '--**-')

('1 из 1:', [['гладь', 726]])

### Отгадываем слово "амбра"

In [52]:
wp.reset() # сбрасываем поиск
wp.predict_next_words('порка', '--*-+') # одну "а" отгадали

('10 из 80:',
 [['сайра', 932],
  ['митра', 895],
  ['раина', 881],
  ['гетра', 876],
  ['руина', 874],
  ['будра', 856],
  ['рента', 856],
  ['среда', 820],
  ['лавра', 793],
  ['махра', 775]])

In [53]:
wp.predict_next_words('сайра', '-*-++') # нашлась вторая "а", в выдаче будут только слова с двумя "а"

('5 из 5:',
 [['тиара', 612],
  ['мшара', 304],
  ['амбра', 252],
  ['афера', 238],
  ['хмара', 217]])

In [54]:
wp.predict_next_words('тиара', '--*++') # для второй "а" осталось место только в начале слова

('2 из 2:', [['амбра', 252], ['афера', 238]])

### Отгадываем слово...

In [58]:
wp.reset() # сбрасываем поиск
wp.predict_next_words('порка', '-**--')

('10 из 64:',
 [['рулон', 1613],
  ['сидор', 1545],
  ['тенор', 1505],
  ['минор', 1467],
  ['мусор', 1463],
  ['резон', 1425],
  ['футор', 1411],
  ['бугор', 1392],
  ['хутор', 1390],
  ['створ', 1334]])

In [59]:
wp.predict_next_words('рулон', '*-*+-')

('1 из 1:', [['ореол', 849]])

In [57]:
wp.predict_next_words('манер', '-+-++')

('4 из 4:', [['лазер', 268], ['шабер', 262], ['чабер', 212], ['шафер', 163]])